In [1]:
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/yhbyhb/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/tensorflow/python/__init__.py", line 49, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/home/yhbyhb/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/home/yhbyhb/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/home/yhbyhb/anaconda3/envs/carnd-term1/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/yhbyhb/anaconda3/envs/carnd-term1/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcudart.so.8.0: cannot open shared object file: No such file or directory


Error impo

In [26]:
lines =[]
with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images =[]
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = '../data/IMG/' +filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)


In [ ]:
model = Sequential()
model.add(Lamda(lambda x: (x/255.0)- 0.5, input_shape =(160, 320, 3)))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [27]:
#model = Sequential()
#model.add(Flatten(input_shape=(160, 320, 3)))
#model.add(Dense(1))

model.compile(loss='mse', optimizer ='adam')
model.fit(X_train, y_train, validation_split =0.2, shuffle = True)
model.save('model.h5')

Train on 2105 samples, validate on 527 samples
Epoch 1/10
2105/2105 [==============================] - 3s - loss: 29485808.3750 - val_loss: 393246.7468
Epoch 2/10
2105/2105 [==============================] - 2s - loss: 50851.9394 - val_loss: 21934.0279
Epoch 3/10
2105/2105 [==============================] - 2s - loss: 9274.0750 - val_loss: 16541.1563
Epoch 4/10
2105/2105 [==============================] - 2s - loss: 5946.1740 - val_loss: 10972.0599
Epoch 5/10
2105/2105 [==============================] - 2s - loss: 4185.3367 - val_loss: 4760.4404
Epoch 6/10
2105/2105 [==============================] - 2s - loss: 3539.8255 - val_loss: 4266.2666
Epoch 7/10
2105/2105 [==============================] - 2s - loss: 3325.1175 - val_loss: 4277.4900
Epoch 8/10
2105/2105 [==============================] - 2s - loss: 2914.7997 - val_loss: 4246.9807
Epoch 9/10
2105/2105 [==============================] - 2s - loss: 2229.2880 - val_loss: 6346.1993
Epoch 10/10
2105/2105 [=============================

In [25]:
(y_train)

array(0.0)